In [27]:
#@title Townscript to Sheet https://docs.google.com/spreadsheets/d/1eWB8KQkfsx8TVib8m8RNlv1PZNgoBcUCBt02lMhAPYA/edit
import requests
import yaml
import pandas as pd
print(pd.Timestamp.now())

2023-09-07 20:27:29.410266


In [29]:
#@title Login Info
from dotenv import load_dotenv
load_dotenv()
cfg=yaml.safe_load("""
api:
  base: https://www.townscript.com/api
apis:
  /user/loginwithtownscript
  /eventdata/get
sheet:
  url: https://docs.google.com/spreadsheets/d/1eWB8KQkfsx8TVib8m8RNlv1PZNgoBcUCBt02lMhAPYA/edit#gid=739860648
  reg: DKD2023_reg
  tickets: DKD2023_tkt
""")

# Code

In [34]:
type(json.loads(os.environ['SERVICE_ACCOUNT']))

dict

In [3]:
from townscript import Townscript
from misc import subDict_tkt, subDict_ans, update_tab

In [4]:
# from google.colab import auth
# auth.authenticate_user()

import gspread
# from google.auth import default
# creds, _ = default()

# gc = gspread.authorize(creds)

" Move the downloaded file to ~/.config/gspread/service_account.json. Windows users should put this file to %APPDATA%\gspread\service_account.json. "
gc = gspread.service_account()

# Download

In [24]:
from dotenv import load_dotenv

load_dotenv() 

True

In [6]:
import json

TS=Townscript(cfg)
TS.TSToken()
event=TS.getEvents('DKD2023')
skipcols=['ticketAndDiscountList','answerList']
# TS.token

In [7]:
dat=TS.getData('DKD2023')

## test

In [8]:

import re, pandas as pd
# df_=pd.DataFrame([(re.search("(\d+)",x)[0],x,dat[0][x]) for x in dat[0] if 'customQ' in x]+
#              [(re.search("\d+",x)[0],x,dat[0][x]) for x in dat[0] if 'customA' in x])
# df_[0]=df_[0].astype(int)
# df_.sort_values(0)

# Upload

In [15]:
gs=gc.open_by_url(cfg['sheet']['url'])
gs_reg=gs.worksheet('DKD2023_reg')
gs_tkt=gs.worksheet('DKD2023_tkt')

In [10]:
"Registration"

df_ans=pd.DataFrame(
                    subDict_ans(dat,
                    subdict='answerList',
                    keys=['uniqueOrderId']))
df_reg=pd.DataFrame(dat).merge(df_ans,how='outer',on='uniqueOrderId')
skipcols_=['answerList','ticketAndDiscountList']
cols_=[c for c in df_reg.columns
 if not (('custom' in c) or ( c in skipcols_))]
print(cols_)
df_reg=df_reg[cols_]


['ticketCurrency', 'registrationId', 'eventName', 'discountCode2', 'discountCode1', 'uniqueOrderId', 'totalTicketAmount', 'ticketPrice1', 'ticketPrice2', 'ticketName2', 'ticketName1', 'userEmailId', 'allTicketName', 'currency', 'userName', 'eventCode', 'registrationTimestamp', 'discountAmount1', 'discountAmount2', 'discountAmount', 'ticketPrice', 'ticketName', 'discountCode', 'discountCode3', 'ticketPrice3', 'ticketName3', 'discountAmount3', 'Contact Number', 'Gender', 'Date Of birth', 'T Shirt option', 'T-shirt size', 'BIB , T Shirt Collection Location', "Name on the T Shirt( type Blank if you don't want to print anything)", 'Emergency Contact Name', 'Emergency Contact Number', 'Pincode/Zipcode/Postcode', 'Your DKD/PKP 2022 BIB', 'Your Society', 'How did you come to know about the event', 'Upload your Photograph where your face is clearly visible and your face is only face.', 'I have read the terms and condition and provide my consent', 'Choose your distance for JUNE-25 LSOM at distri

In [11]:

"Tickets"
import pandas as pd

df_tkt=pd.DataFrame(subDict_tkt(dat,
                     subdict='ticketAndDiscountList',
                     keys=['registrationId','uniqueOrderId','userEmailId',
                           'userName','Contact Number','Gender','T Shirt option',
                           'T-shirt size','BIB , T Shirt Collection Location',
                           "Name on the T Shirt( type Blank if you don't want to print anything)","registrationTimestamp"]))

In [12]:
"show last few registrations"
print('df_reg',df_reg.shape,'Tickets',df_tkt.shape)
showColumns="registrationTimestamp registrationId uniqueOrderId userName ".split()
display(df_reg.sort_values('registrationId',ascending=False).head(2)[showColumns].style)

df_reg (1208, 43) Tickets (1636, 15)


,registrationTimestamp,registrationId,uniqueOrderId,userName
158,20-08-2023 21:46,11555237,664601692548099691,Aniket Gangakhedkar
1049,16-08-2023 10:46,11530141,537401692162958456,Sonali Chaudhari


In [13]:
"show last few tickets"
showColumns="registrationTimestamp registrationId uniqueOrderId userName ticketName ticketPrice".split()
df_tkt[showColumns].sort_values('registrationId',ascending=False).head(10)

,registrationTimestamp,registrationId,uniqueOrderId,userName,ticketName,ticketPrice
223,20-08-2023 21:46,11555237,664601692548099691,Aniket Gangakhedkar,DKD - 2023 - Annual Membership (Register with...,900.0
1438,16-08-2023 10:46,11530141,537401692162958456,Sonali Chaudhari,DKD - 2023 - Annual Membership (Register with...,900.0
1256,11-08-2023 12:22,11478727,450631691736688569,Swasti Swapnil Jadhav,DKD - 2023 - Annual Membership (Register with...,900.0
1194,11-08-2023 12:18,11478704,571271691736427350,Aniket Kumar Shah,DKD - 2023 - Annual Membership (Register with...,900.0
1204,11-08-2023 12:13,11478659,615531691736098071,Anil Kumar Shah,DKD - 2023 - Annual Membership (Register with...,900.0
1326,11-08-2023 11:57,11478555,125221691735119191,sandeep kashid,DKD - 2022 Blue T-shirt (Large Size - unisex ),300.0
1327,11-08-2023 11:57,11478554,125221691735119191,Sandeep Kashid,DKD - 2023 - Annual Membership (Register with...,900.0
1190,07-08-2023 23:33,11441848,721201691431376814,Ajay Dinkar society,DKD - 2022 Blue T-shirt (Large Size - unisex ),300.0
1193,07-08-2023 23:33,11441847,721201691431376814,Ajay Dinkar Suryawanshi,DKD - 2023 - Annual Membership (Register with...,900.0
1557,05-08-2023 11:16,11204370,767131691214336398,Varun,DKD - 2022 Blue T-shirt (Large Size - unisex ),300.0


In [16]:

" update"
update_tab(gs_reg,df_reg)
update_tab(gs_tkt,df_tkt)

Error UndefinedVariableError("local variable 'newKeys' is not defined")
Error UndefinedVariableError("local variable 'newKeys' is not defined")


In [17]:
# gs_reg.row_count
df_old = pd.DataFrame(gs_reg.get_all_records())
df_old.shape, df_reg.shape

((1267, 45), (1208, 43))

In [23]:
df_tkt.pivot_table(columns="BIB , T Shirt Collection Location",
                index=['ticketName'],
                values="ticketPrice",
                aggfunc="count")

"BIB , T Shirt Collection Location",,District Circle - Spine Road,Ganesh Talav - Nigdi,Pavaneshwar Tea stall Chinchwad,Savitri Bai Phule garden - Wakad
ticketName,,,,,
DKD - 2022 Blue T-shirt (Small Size unisex ),NaN,1.0,2.0,NaN,5.0
DKD - 2022 Blue T-shirt (Large Size - unisex ),NaN,10.0,2.0,6.0,10.0
DKD - 2022 Blue T-shirt (Medium Size - unisex ),NaN,4.0,4.0,5.0,2.0
DKD - 2022 Tee Shirt ( Limited stock ),1.0,NaN,NaN,NaN,NaN
DKD - 2023 - Annual Membership (Register with PCMC Runners 2023 T-shirt),NaN,153.0,169.0,94.0,147.0
DKD - 2023 - Annual Membership (Register without T-shirt),NaN,50.0,52.0,53.0,85.0
DKD - 2023 - Annual Membership (Register without T-shirt) with free entry to JUNE-25-2023 LSOM,3.0,NaN,NaN,NaN,NaN
No - Not Interested in Add ons now,NaN,27.0,11.0,10.0,18.0
No Addon,NaN,139.0,157.0,108.0,176.0


In [37]:
## Search

In [26]:
df_tkt.head(3).to_html()

TypeError: to_html() got an unexpected keyword argument 'attr'